# RAG: Better generation

In [./rag_0_indexing.ipynb](./rag_0_indexing.ipynb), we indexed a geography text book.
In [./rag_1_generation.ipynb](./rag_0_generation.ipynb), we used RAG to answer questions grounded by that text book.

In this notebook, we improve the generation.

## Set up.

Install the necessary packages, set up the API keys etc.

In [1]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from dotenv import load_dotenv
from langchain import hub

load_dotenv("../keys.env");

In [2]:
PROVIDER = "Google"
#PROVIDER = "OpenAI"
PERSIST_DIR = "vectordb"

In [3]:
if PROVIDER == "Google":
    from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.1)
else:
    from langchain_openai import ChatOpenAI, OpenAIEmbeddings
    embeddings = OpenAIEmbeddings()
    model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

## Hypothetical answer

We use the RAG to generate an answer to the question, and search the vector store for things that are semantically similar to that answer.

In [4]:
hyp_prompt = """
Answer the following question concisely in 3 sentences or less.
If you don't know the correct answer, provide the most likely answer.

Question:
{question}
"""

hypothetical_answer = (
    {
        "question" : RunnablePassthrough()
    }
    | PromptTemplate.from_template(hyp_prompt)
    | model
    | StrOutputParser()
)
hypothetical_answer.invoke("What types of rock do you find in the Upper Lias?")

'The Upper Lias is a geological formation known for its rich fossil content.  It primarily consists of **claystone and limestone**, with some **shale and sandstone** also present. These rocks were formed in a marine environment during the Jurassic period. \n'

# Build a RAG chain using the hypothetical answer

In [5]:
vectorstore = Chroma(embedding_function=embeddings, persist_directory=PERSIST_DIR)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [6]:
retrieval_chain = hypothetical_answer | retriever
retrieval_chain.invoke("What types of rock do you find in the Upper Lias?")

[Document(metadata={'paragraph': 26, 'source': 'geography'}, page_content='Chapter XX—JURASSIC GROUP, CONTINUED.—LIAS. Mineral Character of Lias. — Numerous successive Zones in the Lias, marked by distinct Fossils, without Unconformity in the Stratification, or Change in the Mineral Character of the Deposits. — Gryphite Limestone. — Shells of the Lias. — Fish of the Lias. — Reptiles of the Lias. — Ichthyosaur and Plesiosaur. — Marine Reptile of the Galapagos Islands. — Sudden Destruction and Burial of Fossil Animals in Lias. — Fluvio-marine Beds in Gloucestershire, and Insect Limestone. — Fossil Plants. — The origin of the Oolite and Lias, and of alternating Calcareous and Argillaceous Formations.'),
 Document(metadata={'paragraph': 1103, 'source': 'geography'}, page_content='Lias.—The English provincial name of Lias has been very generally adopted for a formation of argillaceous limestone, marl, and clay, which forms the base of the Oolite, and is classed by many geologists as part of

In [7]:
def add_docs_to_context(docs):
    return "\n".join(doc.page_content for doc in docs)

In [8]:
rag_prompt = hub.pull("rlm/rag-prompt")
print(rag_prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [9]:
rag_chain = (
    {"context": retrieval_chain | add_docs_to_context, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [10]:
rag_chain.invoke("What types of rock do you find in the Upper Lias?")

'The Upper Lias consists primarily of sands, clay shale, and thin beds of limestone. These layers are characterized by distinct fossils, which help to differentiate them from other Lias formations. The Upper Lias is considered the base of the Oolite by some geologists. \n'

## Rerank the retrieved documents


In [12]:
# add reranking to the retrieval chain
compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)
retrieval_chain = hypothetical_answer | compression_retriever

# rest is the same
rag_chain = (
    {"context": retrieval_chain | add_docs_to_context, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("What types of rock do you find in the Upper Lias?")

'The Upper Lias consists primarily of sands, clay shale, and thin beds of limestone. These formations were once considered part of the Oolite but are now classified as Lias due to their fossil content. The Upper Lias is the uppermost layer of the Lias group, which is a formation of argillaceous limestone, marl, and clay. \n'